# How-To: Test the PyIntact Istari Integration with Python SDK

This notebook walks through a full SDK-based integration test:
1. Connect to Istari with your PAT
2. Create a new system
3. Upload PyIntact STL models
4. Create a system configuration with tracked files
5. Submit `@istari:run_pyintact_simulation`
6. Poll until complete and inspect artifacts

This is designed as a practical how-to before building the full PhysicsNeMo pipeline.

In [ ]:
%pip install -q istari-digital-client python-dotenv

In [ ]:
from __future__ import annotations

import json
import os
import time
from datetime import datetime
from getpass import getpass
from pathlib import Path

from istari_digital_client import Client, Configuration
from istari_digital_client.v2.models.new_system import NewSystem
from istari_digital_client.v2.models.new_system_configuration import NewSystemConfiguration
from istari_digital_client.v2.models.new_tracked_file import NewTrackedFile
from istari_digital_client.v2.models.new_source import NewSource
from istari_digital_client.v2.models.tracked_file_specifier_type import TrackedFileSpecifierType

In [ ]:
def page_items(page_like):
    if page_like is None:
        return []
    if isinstance(page_like, list):
        return page_like
    items = getattr(page_like, "items", None)
    if items is not None:
        return list(items)
    return []


def job_status(job) -> str:
    status = getattr(job, "status", None)
    if status is not None:
        name = getattr(status, "name", None)
        if name is not None:
            return str(getattr(name, "value", name)).lower()
    status_name = getattr(job, "status_name", None)
    if status_name is not None:
        return str(getattr(status_name, "value", status_name)).lower()
    return "unknown"


def latest_revision_id(model) -> str:
    f = getattr(model, "file", None)
    revs = getattr(f, "revisions", []) if f else []
    if not revs:
        raise RuntimeError(f"No revisions found on model {getattr(model, 'id', '<unknown>')}")
    return revs[0].id


def find_models_dir() -> Path:
    candidates = [
        Path.cwd() / "pyintact-integration" / "models",
        Path.cwd().parent / "pyintact-integration" / "models",
    ]
    for c in candidates:
        if c.exists():
            return c
    raise FileNotFoundError("Could not find pyintact-integration/models directory from current working dir")

## Configure Auth

In [ ]:
# Prefer environment variables if already set; otherwise prompt for PAT.
ISTARI_DIGITAL_REGISTRY_URL = os.getenv("ISTARI_DIGITAL_REGISTRY_URL", "https://fileservice-v2.demo.istari.app")
ISTARI_DIGITAL_REGISTRY_AUTH_TOKEN = os.getenv("ISTARI_DIGITAL_REGISTRY_AUTH_TOKEN") or getpass("Enter Istari PAT: ")

AGENT_DISPLAY_NAME = "Istari Slack Analytics Agent"

client = Client(Configuration(
    registry_url=ISTARI_DIGITAL_REGISTRY_URL,
    registry_auth_token=ISTARI_DIGITAL_REGISTRY_AUTH_TOKEN,
))

user = client.get_current_user()
print("Connected as:", getattr(user, "email", "<unknown>"))

In [ ]:
# Confirm function exists and pick agent
fns = page_items(client.list_functions(name="run_pyintact_simulation", size=50))
fn_keys = [getattr(f, "key", None) for f in fns]
print("Function matches:", fn_keys)
assert "@istari:run_pyintact_simulation" in fn_keys, "run_pyintact_simulation function not found in tenant"

agents = page_items(client.list_agents(size=100))
agent = next((a for a in agents if str(getattr(a, "display_name", "")) == AGENT_DISPLAY_NAME), None)
if agent is None:
    agent = agents[0] if agents else None
assert agent is not None, "No agents found in tenant"
print("Using agent:", getattr(agent, "display_name", None), getattr(agent, "id", None))

## Create System + Upload Models

In [ ]:
ts = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
system_name = f"PyIntact SDK HowTo {ts}"

system = client.create_system(NewSystem(
    name=system_name,
    description="SDK how-to system for pyintact integration validation",
))

models_dir = find_models_dir()
beam_path = models_dir / "beam.stl"
load_path = models_dir / "end-2.stl"
restraint_path = models_dir / "end-1.stl"

beam_model = client.add_model(path=beam_path, display_name=f"pyintact-beam-{ts}")
load_model = client.add_model(path=load_path, display_name=f"pyintact-load-face-{ts}")
restraint_model = client.add_model(path=restraint_path, display_name=f"pyintact-restraint-face-{ts}")

print("System ID:", system.id)
print("Beam model ID:", beam_model.id)
print("Load model ID:", load_model.id)
print("Restraint model ID:", restraint_model.id)

In [ ]:
# Create configuration and track uploaded files
tracked = [
    NewTrackedFile(specifier_type=TrackedFileSpecifierType.LATEST, file_id=beam_model.file.id),
    NewTrackedFile(specifier_type=TrackedFileSpecifierType.LATEST, file_id=load_model.file.id),
    NewTrackedFile(specifier_type=TrackedFileSpecifierType.LATEST, file_id=restraint_model.file.id),
]

cfg = client.create_configuration(
    system_id=system.id,
    new_system_configuration=NewSystemConfiguration(
        name=f"pyintact-howto-config-{ts}",
        tracked_files=tracked,
    ),
)

print("Configuration ID:", cfg.id)

## Submit `@istari:run_pyintact_simulation` Job

Important pattern for this function:
- Use `beam_model.id` as the main `model_id` (unnamed user model input)
- Pass named `NewSource` entries for `load_geometry_file` and `restraint_geometry_file`
- Pass `simulation_config` as a JSON string parameter

In [ ]:
sim_config = {
    "simulation_type": "linear_elastic",
    "material": {
        "density": 7800.0,
        "poisson_ratio": 0.3,
        "youngs_modulus": 2.1e11,
    },
    "load": {
        "type": "pressure",
        "magnitude": 1e6,
    },
    "scenario": {
        "resolution": 1000,
        "units": "MKS",
    },
}

job = client.add_job(
    model_id=beam_model.id,
    function="@istari:run_pyintact_simulation",
    assigned_agent_id=agent.id,
    sources=[
        NewSource(revision_id=latest_revision_id(load_model), relationship_identifier="load_geometry_file"),
        NewSource(revision_id=latest_revision_id(restraint_model), relationship_identifier="restraint_geometry_file"),
    ],
    parameters={
        "simulation_config": json.dumps(sim_config)
    },
    description="PyIntact SDK how-to test job",
    display_name=f"pyintact-howto-job-{ts}",
)

print("Job ID:", job.id)

In [ ]:
# Poll until terminal status
terminal = {"completed", "failed", "cancelled", "canceled", "timed_out", "timeout"}

last = None
for _ in range(120):
    current = client.get_job(job.id)
    s = job_status(current)
    if s != last:
        print("Status:", s)
        last = s
    if s in terminal:
        break
    time.sleep(10)

final_job = client.get_job(job.id)
final_status = job_status(final_job)
print("Final status:", final_status)
assert final_status == "completed", f"Job did not complete successfully: {final_status}"

In [ ]:
# Inspect resulting artifacts on the beam model
artifacts_page = client.list_model_artifacts(model_id=beam_model.id, size=50)
artifacts = page_items(artifacts_page)

print(f"Artifacts found: {len(artifacts)}")
for a in artifacts[:20]:
    file_obj = getattr(a, "file", None)
    revs = getattr(file_obj, "revisions", []) if file_obj else []
    rev_name = getattr(revs[0], "name", None) if revs else None
    print("-", getattr(a, "id", None), "::", rev_name)

## Next Step

Once this test passes, you can scale to the full pipeline:
1. campaign creation + batch submission
2. dataset assembly job
3. PhysicsNeMo training job
4. quality gates and model promotion